<a href="https://colab.research.google.com/github/PradeepKumarLaghavarapu-DS/PRADEEPKUMAR_INFO5731_-Fall2022/blob/main/In_class_exercise/In_class_exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 11/03/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [4]:
# Write your code here

import nltk
nltk.download('stopwords')




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
pip install pyLDAvis==2.1.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 6.6 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=2f88cad1ac8d7976587b96345e8db288a5ae95666b5cb8941ba4a334e8dc5b80
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


In [6]:
pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
#pip install spacy
!python -m spacy download en_core_web_sm

2022-11-07 05:43:17.952026: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 7.1 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
#importing Packages
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
#import en_core_web_sm

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [9]:
#Preparing NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [10]:
# Importing Dataset
df = pd.read_csv('training.csv')

df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [11]:
labels_dict = {0:'sadness',1:'joy',2:'love',3:'anger',4:'fear',5:'surprise'}
df['description'] = df['label'].map(labels_dict)
df.head

<bound method NDFrame.head of                                                     text  label description
0                                i didnt feel humiliated      0     sadness
1      i can go from feeling so hopeless to so damned...      0     sadness
2       im grabbing a minute to post i feel greedy wrong      3       anger
3      i am ever feeling nostalgic about the fireplac...      2        love
4                                   i am feeling grouchy      3       anger
...                                                  ...    ...         ...
15995  i just had a very brief time in the beanbag an...      0     sadness
15996  i am now turning and i feel pathetic that i am...      0     sadness
15997                     i feel strong and good overall      1         joy
15998  i feel like this was such a rude comment and i...      3       anger
15999  i know a lot but i feel so stupid because i ca...      0     sadness

[16000 rows x 3 columns]>

In [12]:
# Converting to list
data = df.text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

['i didnt feel humiliated']


In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['didnt', 'feel', 'humiliated']]


In [14]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['didnt', 'feel', 'humiliated']


In [15]:
# Defining functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm", disable=['parser','ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['feel', 'humiliate']]


In [17]:
# Creating Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1)]]


In [18]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('feel', 1), ('humiliate', 1)]]

In [19]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [20]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.046*"even" + 0.036*"good" + 0.031*"look" + 0.029*"back" + 0.024*"friend" '
  '+ 0.015*"world" + 0.014*"last" + 0.012*"wake" + 0.012*"often" + '
  '0.012*"like"'),
 (1,
  '0.104*"bit" + 0.070*"help" + 0.036*"glad" + 0.035*"week" + 0.032*"ask" + '
  '0.028*"move" + 0.023*"support" + 0.019*"torture" + 0.016*"depressed" + '
  '0.015*"great"'),
 (2,
  '0.135*"think" + 0.047*"give" + 0.046*"s" + 0.037*"hate" + 0.026*"moment" + '
  '0.019*"watch" + 0.019*"night" + 0.018*"ill" + 0.018*"game" + 0.017*"fine"'),
 (3,
  '0.134*"time" + 0.078*"ve" + 0.052*"try" + 0.051*"find" + 0.048*"also" + '
  '0.040*"lot" + 0.031*"keep" + 0.026*"bad" + 0.024*"alone" + 0.022*"point"'),
 (4,
  '0.197*"go" + 0.131*"want" + 0.037*"home" + 0.034*"let" + 0.031*"care" + '
  '0.026*"accept" + 0.022*"sad" + 0.021*"month" + 0.012*"easy" + '
  '0.012*"hopeless"'),
 (5,
  '0.093*"come" + 0.044*"part" + 0.036*"stop" + 0.036*"run" + 0.032*"begin" + '
  '0.026*"experience" + 0.026*"wait" + 0.021*"hot" + 0.019*"cold"

In [21]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.32548228155158

Coherence Score:  0.4253646521671636


In [22]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.451352  0.125130       1        1  18.249038
0      0.103862 -0.424495       2        1   9.558144
18     0.024815 -0.008118       3        1   5.957422
7     -0.004969  0.011472       4        1   5.186228
3      0.002538  0.001729       5        1   5.163468
15    -0.017578  0.013660       6        1   4.505330
17    -0.017503  0.013472       7        1   4.409519
2     -0.032229  0.019162       8        1   4.216573
11    -0.031026  0.018964       9        1   4.205873
12    -0.027328  0.015739      10        1   4.154253
4     -0.023100  0.013818      11        1   4.061429
6     -0.024362  0.017921      12        1   4.005799
19    -0.037045  0.018719      13        1   3.643461
10    -0.046367  0.020512      14        1   3.571679
16    -0.048766  0.023714      15        1   3.528449
1     -0.048737  0.022632      16        1   3.444148
9     -0.050846  0.024453      17        1   3.422794
14    -0.054222  0.023373      18        1   3.063136
5     -0.056074  0.024031      19        1   2.937248
13    -0.062416  0.024113      20        1   2.716011, topic_info=            Term          Freq         Total Category  logprob  loglift
0           feel  15763.000000  15763.000000  Default  30.0000  30.0000
10             m   2379.000000   2379.000000  Default  29.0000  29.0000
127      feeling   1148.000000   1148.000000  Default  28.0000  28.0000
175          get   1106.000000   1106.000000  Default  27.0000  27.0000
43          make   1223.000000   1223.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
2360     produce     30.559189     31.433209  Topic20  -4.7673   3.5778
205      contact     30.259597     31.133617  Topic20  -4.7771   3.5775
495      present     27.173527     28.047547  Topic20  -4.8847   3.5743
3754    facebook     27.032152     27.906172  Topic20  -4.8899   3.5742
435   worthwhile     25.973190     26.847210  Topic20  -4.9299   3.5729

[630 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
996      12  0.990304     ability
125      15  0.995104        able
973      17  0.988959  absolutely
900      10  0.985387       abuse
201      11  0.994463      accept
...     ...       ...         ...
13        7  0.989444       wrong
48       10  0.996837        year
1536     15  0.990415   yesterday
594       2  0.991891         yet
362      13  0.960816       young

[600 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 1, 19, 8, 4, 16, 18, 3, 12, 13, 5, 7, 20, 11, 17, 2, 10, 15, 6, 14])

In [23]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

--2022-11-07 05:46:25--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip [following]
--2022-11-07 05:46:26--  https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  16.6MB/s    in 0.9s    

2022-11-07 05:46:27 (16.6 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]



In [24]:
!unzip mallet-2.0.8.zip

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: mallet-2.0.8/bin/svmlight2vectors  
  inflating: mallet-2.0.8/bin/text2classify  
  inflating: mallet-2.0.8/bin/text2vectors  
  inflating: mallet-2.0.8/bin/vectors2classify  
  inflating: mallet-2.0.8/bin/vectors2info  
  inflating: mallet-2.0.8/bin/vectors2topics  
  inflating: mallet-2.0.8/bin/vectors2vectors  
  inflating: mallet-2.0.8/build.xml  
   creating: mallet-2.0.8/class/
   creating: mallet-2.0.8/class/cc/
   creating: mallet-2.0.8/class/cc/mallet/
   creating: mallet-2.0.8/class/cc/mallet/classify/
  inflating: mallet-2.0.8/class/cc/mallet/classify/AdaBoost.class  
  in

In [25]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
from gensim import utils, matutils
from gensim.models import basemodel
#from gensim.models.wrappers import LdaMallet
from gensim.models.ldamodel import LdaModel
from gensim.utils import check_output, revdict

In [27]:
# mallet_path = "/content/mallet-2.0.8/bin/mallet"
# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [28]:
# Write your code here

import pandas as pd

# load data
df = pd.read_csv('training.csv')
df.head()


,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [29]:
labels_dict = {0:'sadness',1:'joy',2:'love',3:'anger',4:'fear',5:'surprise'}
df['description'] = df['label'].map(labels_dict)
df.head

<bound method NDFrame.head of                                                     text  label description
0                                i didnt feel humiliated      0     sadness
1      i can go from feeling so hopeless to so damned...      0     sadness
2       im grabbing a minute to post i feel greedy wrong      3       anger
3      i am ever feeling nostalgic about the fireplac...      2        love
4                                   i am feeling grouchy      3       anger
...                                                  ...    ...         ...
15995  i just had a very brief time in the beanbag an...      0     sadness
15996  i am now turning and i feel pathetic that i am...      0     sadness
15997                     i feel strong and good overall      1         joy
15998  i feel like this was such a rude comment and i...      3       anger
15999  i know a lot but i feel so stupid because i ca...      0     sadness

[16000 rows x 3 columns]>

In [30]:
print(df.shape)
df = df.dropna()
print(df.shape)

(16000, 3)
(16000, 3)


In [31]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
                                        , preprocess_string, strip_short, stem_text

# preprocess given text
def preprocess(text):
    
    # clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(), 
                                remove_stopwords, 
                                strip_punctuation, 
                                strip_short, 
                                stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)
    
    return text

# apply function to all reviews 
df['text (Clean)'] = df['text'].apply(lambda x: preprocess(x))

In [32]:
from gensim import corpora

# create a dictionary with the corpus
corpus = df['text (Clean)']
dictionary = corpora.Dictionary(corpus)

# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

In [33]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df['text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 2 clusters: 0.31062520094361007
Coherence score with 3 clusters: 0.35508076017583895
Coherence score with 4 clusters: 0.36427157545085853
Coherence score with 5 clusters: 0.3529205542432569
Coherence score with 6 clusters: 0.32092721506214994
Coherence score with 7 clusters: 0.34815763156570684
Coherence score with 8 clusters: 0.35460909076097497
Coherence score with 9 clusters: 0.3513594578454482
Coherence score with 10 clusters: 0.3179511930076909


In [34]:
lsi = LsiModel(bow, num_topics=10, id2word=dictionary)

for topic_num, words in lsi.print_topics(num_words=5):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.954*"feel" + 0.199*"like" + 0.057*"time" + 0.054*"know" + 0.051*"want".
Words in 1: -0.974*"like" + 0.204*"feel" + -0.025*"thing" + -0.020*"dont" + 0.019*"littl".
Words in 2: 0.640*"know" + 0.421*"time" + 0.367*"want" + 0.269*"love" + 0.223*"peopl".
Words in 3: 0.848*"time" + -0.334*"want" + -0.329*"know" + -0.125*"dont" + 0.095*"iv".
Words in 4: 0.843*"love" + -0.443*"know" + 0.180*"want" + 0.127*"peopl" + -0.096*"time".
Words in 5: -0.800*"want" + 0.416*"know" + 0.363*"love" + -0.165*"time" + 0.059*"dai".
Words in 6: -0.693*"littl" + -0.550*"dai" + -0.264*"thing" + -0.191*"bit" + -0.172*"iv".
Words in 7: 0.707*"think" + 0.371*"thing" + -0.331*"dai" + 0.308*"peopl" + -0.148*"love".
Words in 8: 0.691*"dai" + -0.594*"littl" + 0.194*"think" + -0.150*"bit" + 0.120*"peopl".
Words in 9: 0.827*"thing" + -0.402*"think" + -0.239*"littl" + -0.181*"peopl" + -0.128*"dai".


## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [35]:
pip install lda2vec==0.16.10

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for lda2vec: filename=lda2vec-0.16.10-py3-none-any.whl size=14433 sha256=748dd0cbf2767e6d05a86f922cff55884d76fa4b03b53692f0e4fec1632aad49
  Stored in directory: /root/.cache/pip/wheels/76/51/1c/1e2960ded8d6e14b14454493fcc9c2d879a7fb99421867f177
Successfully built lda2vec


In [36]:
#!pip install tensorflow

In [51]:
# Write your code here

!from lda2vec import preprocess, Corpus
import matplotlib.pyplot as plt
#import tensorflow as tf
import numpy as np
%matplotlib inline

try:
    import seaborn
except:
    pass



from: can't read /var/mail/lda2vec


In [38]:
import pyLDAvis
pyLDAvis.enable_notebook()

In [39]:
npz = np.load(open('topics.pyldavis.npz', 'r'))
dat = {k: v for (k, v) in npz.iteritems()}
dat['vocab'] = dat['vocab'].tolist()


top_n = 10
topic_to_topwords = {}
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [dat['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print(msg)
    topic_to_topwords[j] = top_words

FileNotFoundError: ignored

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [40]:
# Write your code here

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('Tokyo Medals 2021.csv')
df = df.dropna()
df = df.sample(20, random_state=42)
df.head()



,Country,Gold Medal,Silver Medal,Bronze Medal,Total,Rank By Total
40,Qatar,2,0,1,3,60
22,Sweden,3,6,0,9,26
55,Ethiopia,1,1,2,4,47
72,San Marino,0,1,2,3,60
0,United States of America,39,41,33,113,1


In [41]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x


df['clean_text'] = df.Country.apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Country,Gold Medal,Silver Medal,Bronze Medal,Total,Rank By Total,clean_text
40,Qatar,2,0,1,3,60,qatar
22,Sweden,3,6,0,9,26,sweden
55,Ethiopia,1,1,2,4,47,ethiopia
72,San Marino,0,1,2,3,60,san marino
0,United States of America,39,41,33,113,1,united states america


In [42]:
!python -m pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [45]:
from bertopic import BERTopic
tweets = df.clean_text.to_list()
timestamp = df.Country.to_list()

topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(tweets)

topic_model.get_topic_info()

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

,Topic,Count,Name
0,-1,20,-1_republic_israel_kenya_united


## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.




From the above results, we see that Bertopic results are easily interpretable.
